# (목적)Gradient를 확인한다
## 전체 Gradient에 3을 곱한것과 parameter 별로 gradient에 3을 곱한 것이 차이가 나는지 확인

In [16]:
import torch
import torch.nn as nn

import torch.nn.functional as F

# 예제 데이터 생성
X = torch.randn(100, 2)
y = torch.randint(0, 2, (100,))

# 모델 정의
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 2)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

model = ClassificationModel() # without gradient modifier
model_with_modifier = ClassificationModel()

# Cross Entropy Loss와 KL Divergence Loss 정의
cross_entropy_loss = nn.CrossEntropyLoss()
kl_divergence_loss = nn.KLDivLoss(reduction='batchmean')

# Forward Pass
outputs = model(X)
output_with_modifier = model_with_modifier(X)

# Cross Entropy Loss 계산
ce_loss = F.cross_entropy(outputs, y)

# KL Divergence Loss 계산
p = torch.tensor([0.1, 0.2])  # 예시 분포
kl_loss = kl_divergence_loss(torch.log(outputs), p)
# kl_loss = ce_loss

# Gradient 계산 (torch.autograd.grad 사용)
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
kl_gradient = torch.autograd.grad(kl_loss, model.parameters(), retain_graph=True)

# Gradient 확인
print("Cross Entropy Loss Gradient:")
for grad_param in ce_gradient:
    print(grad_param)

# print("\nKL Divergence Loss Gradient:")
# for grad_param in kl_gradient:
#     print(grad_param)

Cross Entropy Loss Gradient:
tensor([[ 0.0327, -0.0238],
        [-0.0174,  0.0127]])
tensor([-0.0167,  0.0089])
tensor([[ 0.0453, -0.0398],
        [-0.0453,  0.0398]])
tensor([-0.0326,  0.0326])


In [17]:
ce_loss *= 2
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)

# Gradient 확인
print("Cross Entropy Loss Gradient:")
for grad_param in ce_gradient:
    print(grad_param)

Cross Entropy Loss Gradient:
tensor([[ 0.0655, -0.0477],
        [-0.0348,  0.0253]])
tensor([-0.0334,  0.0178])
tensor([[ 0.0906, -0.0796],
        [-0.0906,  0.0796]])
tensor([-0.0653,  0.0653])


In [18]:
kl_loss

tensor(nan, grad_fn=<DivBackward0>)

In [19]:
total_loss = 1 * ce_loss +  2* kl_loss
total_loss
total_gradient = torch.autograd.grad(total_loss, model.parameters(), retain_graph=True)
print(total_gradient)

(tensor([[-1.7330, -2.9492],
        [-4.5699, -5.9226]]), tensor([-1.3262, -4.2677]), tensor([[ 0.0880, -0.6988],
        [-0.8494, -0.1448]]), tensor([-7.1931, -1.8057]))


In [20]:
names_grads_copy = {}

for param_name, ce_grad, kl_grad in zip(model.parameters(), ce_gradient, kl_gradient):
    
    #kl_grad = None
    
    if not kl_grad == None:
        names_grads_copy[param_name] = torch.tensor(1) * ce_grad + torch.tensor(2) *  kl_grad
    else:
        names_grads_copy[param_name] = torch.tensor(1) * ce_grad 
    
for key, value in names_grads_copy.items():
    print(value)

tensor([[-1.7330, -2.9492],
        [-4.5699, -5.9226]])
tensor([-1.3262, -4.2677])
tensor([[ 0.0880, -0.6988],
        [-0.8494, -0.1448]])
tensor([-7.1931, -1.8057])


## 1. Gradient를 추가적으로 구해도 retain_graph=True를 설정하면, 값이 변하지 않는다

In [21]:
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[ 0.0655, -0.0477],
        [-0.0348,  0.0253]])
tensor([-0.0334,  0.0178])
tensor([[ 0.0906, -0.0796],
        [-0.0906,  0.0796]])
tensor([-0.0653,  0.0653])


# 2. 전체 gradient에 3을 곱한다
## 1) 안된다

In [22]:
ce_gradient =  torch.tensor(3) * torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[ 0.0655, -0.0477],
        [-0.0348,  0.0253]])
tensor([-0.0334,  0.0178])
tensor([[ 0.0906, -0.0796],
        [-0.0906,  0.0796]])
tensor([-0.0653,  0.0653])
tensor([[ 0.0655, -0.0477],
        [-0.0348,  0.0253]])
tensor([-0.0334,  0.0178])
tensor([[ 0.0906, -0.0796],
        [-0.0906,  0.0796]])
tensor([-0.0653,  0.0653])
tensor([[ 0.0655, -0.0477],
        [-0.0348,  0.0253]])
tensor([-0.0334,  0.0178])
tensor([[ 0.0906, -0.0796],
        [-0.0906,  0.0796]])
tensor([-0.0653,  0.0653])


## 2) 덧셈 조차 하면 안된다

In [23]:
total_gradient = ce_gradient + kl_gradient
total_gradient

(tensor([[ 0.0655, -0.0477],
         [-0.0348,  0.0253]]),
 tensor([-0.0334,  0.0178]),
 tensor([[ 0.0906, -0.0796],
         [-0.0906,  0.0796]]),
 tensor([-0.0653,  0.0653]),
 tensor([[ 0.0655, -0.0477],
         [-0.0348,  0.0253]]),
 tensor([-0.0334,  0.0178]),
 tensor([[ 0.0906, -0.0796],
         [-0.0906,  0.0796]]),
 tensor([-0.0653,  0.0653]),
 tensor([[ 0.0655, -0.0477],
         [-0.0348,  0.0253]]),
 tensor([-0.0334,  0.0178]),
 tensor([[ 0.0906, -0.0796],
         [-0.0906,  0.0796]]),
 tensor([-0.0653,  0.0653]),
 tensor([[-0.8993, -1.4507],
         [-2.2676, -2.9740]]),
 tensor([-0.6464, -2.1427]),
 tensor([[-0.0013, -0.3096],
         [-0.3794, -0.1122]]),
 tensor([-3.5639, -0.9355]))

## 3) loss에 3을 곱해줘야 내가 원하는 값을 얻을 수 있다

In [24]:
ce_gradient =  torch.autograd.grad(torch.tensor(3) * ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[ 0.1964, -0.1430],
        [-0.1044,  0.0760]])
tensor([-0.1002,  0.0533])
tensor([[ 0.2717, -0.2389],
        [-0.2717,  0.2389]])
tensor([-0.1958,  0.1958])


# 3. 각 parameter 별로 3을 곱해준다

In [25]:
# Gradient 확인
print("Cross Entropy Loss Gradient:")
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(torch.tensor(3) * grad_param)

Cross Entropy Loss Gradient:
tensor([[ 0.1964, -0.1430],
        [-0.1044,  0.0760]])
tensor([-0.1002,  0.0533])
tensor([[ 0.2717, -0.2389],
        [-0.2717,  0.2389]])
tensor([-0.1958,  0.1958])


## 4. weight에 2를 곱하고 gradient를 구하면 값이 달라진다

In [26]:
# 각 레이어의 가중치에 2를 곱함
for param in model.parameters():
    param.data = param.data * 2

ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[ 0.0655, -0.0477],
        [-0.0348,  0.0253]])
tensor([-0.0334,  0.0178])
tensor([[ 0.0906, -0.0796],
        [-0.0906,  0.0796]])
tensor([-0.0653,  0.0653])


# 5. 각 parameter 별로 3을 더해준다

In [27]:
# Gradient 확인
print("Cross Entropy Loss Gradient:")
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

Cross Entropy Loss Gradient:
tensor([[ 0.0655, -0.0477],
        [-0.0348,  0.0253]])
tensor([-0.0334,  0.0178])
tensor([[ 0.0906, -0.0796],
        [-0.0906,  0.0796]])
tensor([-0.0653,  0.0653])


In [28]:
# Gradient 확인
print("Cross Entropy Loss Gradient:")
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(torch.tensor(3) + grad_param)

Cross Entropy Loss Gradient:
tensor([[3.0655, 2.9523],
        [2.9652, 3.0253]])
tensor([2.9666, 3.0178])
tensor([[3.0906, 2.9204],
        [2.9094, 3.0796]])
tensor([2.9348, 3.0652])


## 6. Gradient를 조작했을 떄 뭐가 달라지나?

In [29]:
learning_rate = 0.01
with torch.no_grad():
    for param, grad in zip(model.parameters(), ce_gradient):
        param -= learning_rate * grad
        print(param)

Parameter containing:
tensor([[ 0.9868, -1.0277],
        [-0.7411,  0.2353]], requires_grad=True)
Parameter containing:
tensor([0.6279, 0.6512], requires_grad=True)
Parameter containing:
tensor([[ 0.4993,  0.8401],
        [-0.5226,  1.3826]], requires_grad=True)
Parameter containing:
tensor([-0.0163,  0.4733], requires_grad=True)


In [30]:
# learning_rate = 0.01
# with torch.no_grad():
#     for param, grad in zip(model.parameters(), ce_gradient):
#         param -= learning_rate * grad
#         print(param)